In [1]:
model_name = 'distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european'

In [2]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nestorojedagonzalez (nojeda) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset
ds = load_dataset("nojedag/financial_phrasebank_multilingual")

In [4]:
from utils import transform_labels
dataset = ds.map(transform_labels)

In [5]:
# let's train a Distilbert model

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['sentence'], padding='max_length')    

In [6]:
dataset = dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/12268 [00:00<?, ? examples/s]

Map:   0%|          | 0/5258 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)

In [9]:
from transformers import TrainingArguments
from utils import get_output_dir
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  eval_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir=get_output_dir(model_name),
                                  report_to="wandb"
                                )

In [10]:
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['test'].shuffle(seed=10)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [12]:
# Launch the learning process: training
# Load model from checkpoint
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
500,No log,0.878736
1000,No log,0.847984
1500,No log,0.823925
2000,0.883700,0.795764
2500,0.883700,0.827510
3000,0.883700,0.728557
3500,0.767500,0.736863
4000,0.767500,0.685882
4500,0.767500,0.686377
5000,0.587100,0.754772


TrainOutput(global_step=10738, training_loss=0.49595067468429327, metrics={'train_runtime': 3926.3062, 'train_samples_per_second': 21.872, 'train_steps_per_second': 2.735, 'total_flos': 1.1375973197697024e+16, 'train_loss': 0.49595067468429327, 'epoch': 7.0})

In [13]:
from utils import compute_metrics

trainer_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [14]:
trainer_eval.evaluate()

{'eval_loss': 0.6858820915222168,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': {'accuracy': 0.7073031570939521},
 'eval_f1score': {'f1': 0.697143700320662},
 'eval_runtime': 26.7098,
 'eval_samples_per_second': 196.857,
 'eval_steps_per_second': 24.635}

In [15]:
model.save_pretrained(model_name)
model.push_to_hub(f'nojedag/{model_name}')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european/commit/4fdbfd4c2c865af0c785e8656f54d750250ea410', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='4fdbfd4c2c865af0c785e8656f54d750250ea410', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european'), pr_revision=None, pr_num=None)

In [16]:
trainer.push_to_hub()
trainer_eval.push_to_hub()

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european/commit/c83ba7ad76a6344d406bad85b96ae4b6af25aff3', commit_message='End of training', commit_description='', oid='c83ba7ad76a6344d406bad85b96ae4b6af25aff3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/distilbert-base-uncased-finetuned-financial-news-sentiment-analysis-european'), pr_revision=None, pr_num=None)

In [17]:
wandb.finish()

eval/loss,▅▄▄▃▄▂▂▁▁▂▂▂▄▅▄▅▆▅▇██▁
eval/model_preparation_time,▁
eval/runtime,▂▂▃▂▃▂▂▁▂▁▁▁▁▁▂▂▁▁▂▂▂█
eval/samples_per_second,▇▇▆▇▆▇▇█▇█████▇▇██▇▇▇▁
eval/steps_per_second,▇▇▆▇▆▇▇█▇█████▇▇██▇▇▇▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████▁
train/grad_norm,▃▆█▆▁▂▁
train/learning_rate,▇█▇▅▄▂▁
train/loss,█▇▅▃▂▂▁
eval/loss,0.68588
